# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
#! explorer ..

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_file = pd.read_csv("city_data2.csv", dtype="object", encoding="utf-8")
city_data_file.head()
city_data_file.columns
city_data=city_data_file.drop('Unnamed: 0', axis=1)
city_data.head()

,Name,lat,long,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,Cabo San Lucas,22.89,-109.91,28.89,83,5,1603246391,5.7,MX
1,Sitka,57.05,-135.33,6.0,75,75,1603246656,1.58,US
2,East London,-33.02,27.91,14.16,66,0,1603246656,2.46,ZA
3,Sidi Ali,36.1,0.42,21.62,28,100,1603246656,7.56,DZ
4,Bathsheba,13.22,-59.52,28.0,78,40,1603246656,3.6,BB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
location = city_data[["lat", "long"]].astype(float)
humidity=city_data["Humidity"].astype(float)

fig1 = gmaps.figure()
humidity_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
#Add layer
fig1.add_layer(humidity_layer)
# Display figure
fig1



Figure(layout=FigureLayout(height='420px'))

In [19]:
#Please note the temp is in Celsius so I converted it the value which are equivalent to fahrenheit which is 2

Per_weather=city_data.loc[(city_data["Max Temp"].astype(float) <= 26.66) & (city_data["Max Temp"].astype(float) >= 21.11) \
                                   & (city_data["Wind Speed"].astype(float) < 10) \
                                    & (city_data["Cloudiness"].astype(float) == 0)].dropna()

Per_weather.head()


,Name,lat,long,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
14,Busselton,-33.65,115.33,21.67,38,0,1603246657,2.81,AU
22,São Miguel do Iguaçu,-25.35,-54.24,25.0,73,0,1603246658,1.0,BR
25,Grand Gaube,-20.01,57.66,23.89,72,0,1603246659,1.34,MU
41,Alta Floresta,-9.88,-56.09,25.41,62,0,1603246660,0.16,BR
53,Arraial do Cabo,-22.97,-42.03,22.48,85,0,1603246539,9.26,BR


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = Per_weather[["Name","lat","long","Country"]]
hotel_df.set_index('Name')
hotel_df['Nearby Hotel'] = ""
hotel_df

C:\Users\sethi\Anaconda3\envs\pydata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Name,lat,long,Country,Nearby Hotel
14,Busselton,-33.65,115.33,AU,
22,São Miguel do Iguaçu,-25.35,-54.24,BR,
25,Grand Gaube,-20.01,57.66,MU,
41,Alta Floresta,-9.88,-56.09,BR,
53,Arraial do Cabo,-22.97,-42.03,BR,
101,Angoche,-16.23,39.91,MZ,
104,Masunga,-20.62,27.45,BW,
167,Moreira Sales,-24.06,-53.01,BR,
181,Doba,8.65,16.85,TD,
191,Sharjah city,25.36,55.4,AE,


In [21]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Nearby Hotel"] = hotel_name["results"][0]["name"]
        print(f"Hotel Found....") 
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")
    

C:\Users\sethi\Anaconda3\envs\pydata\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\sethi\Anaconda3\envs\pydata\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Missing field/result... skipping.
Missing field/result... skipping.
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....
Hotel Found....


In [22]:
hotel_df

,Name,lat,long,Country,Nearby Hotel
14,Busselton,-33.65,115.33,AU,Cape Lodge
22,São Miguel do Iguaçu,-25.35,-54.24,BR,Belmond Hotel das Cataratas
25,Grand Gaube,-20.01,57.66,MU,LUX Grand Gaube Resort & Villas
41,Alta Floresta,-9.88,-56.09,BR,Cristalino Lodge
53,Arraial do Cabo,-22.97,-42.03,BR,Apa Pau Brasil
101,Angoche,-16.23,39.91,MZ,Guest House HESADA
104,Masunga,-20.62,27.45,BW,Tantebane Resort
167,Moreira Sales,-24.06,-53.01,BR,HOTEL CAIUÁ
181,Doba,8.65,16.85,TD,Olivier
191,Sharjah city,25.36,55.4,AE,Atana Hotel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations1 = hotel_df[["lat", "long"]].astype(float)
#hotel_info

In [24]:
Pf_weather_layer = gmaps.marker_layer(locations1, info_box_content = hotel_info)
#fig2 = gmaps.figure()
fig1.add_layer(Pf_weather_layer)

fig1

Figure(layout=FigureLayout(height='420px'))